In [21]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from collections import defaultdict

In [22]:
df1 = pd.read_csv('datatraining.txt', header=0, parse_dates=[0], index_col=0, squeeze=True)
df2= pd.read_csv('datatest.txt', header=0, parse_dates=[0], index_col=0, squeeze=True)
df3= pd.read_csv('datatest2.txt', header=0, parse_dates=[0], index_col=0, squeeze=True)

df4= pd.merge(df2, df1, how='outer')
df_m=pd.merge(df4, df3, how='outer')

df = df_m.set_index(['date'])

In [23]:
def prepare_dataset(df, class_name):
    df = remove_missing_values(df)
    numeric_columns = get_numeric_columns(df)
    rdf = df.copy(deep=True)
    df, feature_names, class_values = one_hot_encoding(df, class_name)
    real_feature_names = get_real_feature_names(rdf, numeric_columns, class_name)
    rdf = rdf[real_feature_names + (class_values if isinstance(class_name, list) else [class_name])]
    features_map = get_features_map(feature_names, real_feature_names)

    return df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map

def remove_missing_values(df):
    for column_name, nbr_missing in df.isna().sum().to_dict().items():
        if nbr_missing > 0:
            if column_name in df._get_numeric_data().columns:
                mean = df[column_name].mean()
                df[column_name].fillna(mean, inplace=True)
            else:
                mode = df[column_name].mode().values[0]
                df[column_name].fillna(mode, inplace=True)
    return df

def get_numeric_columns(df):
    numeric_columns = list(df._get_numeric_data().columns)
    return numeric_columns

def get_real_feature_names(rdf, numeric_columns, class_name):
    real_feature_names = [c for c in rdf.columns if c in numeric_columns and c != class_name]
    real_feature_names += [c for c in rdf.columns if c not in numeric_columns and c != class_name]
    return real_feature_names

def one_hot_encoding(df, class_name):
    dfX = pd.get_dummies(df[[c for c in df.columns if c != class_name]], prefix_sep='=')
    class_name_map = {v: k for k, v in enumerate(sorted(df[class_name].unique()))}
    dfY = df[class_name].map(class_name_map)
    df = pd.concat([dfX, dfY], axis=1)
    feature_names = list(dfX.columns)
    class_values = sorted(class_name_map)
    return df, feature_names, class_values

def get_features_map(feature_names, real_feature_names):
    features_map = defaultdict(dict)
    i = 0
    j = 0

    while i < len(feature_names) and j < len(real_feature_names):
        if feature_names[i] == real_feature_names[j]:
            features_map[j][feature_names[i]] = j
            i += 1
            j += 1
        elif feature_names[i].startswith(real_feature_names[j]):
            features_map[j][feature_names[i]] = j
            i += 1
        else:
            j += 1
    return features_map

In [24]:
class_name = 'Occupancy'
res = prepare_dataset(df, class_name)
df, feature_names, class_values, numeric_columns, rdf, real_feature_names, features_map = res
df.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
date,,,,,,
2015-02-02 14:19:00,23.7000,26.272,585.200000,749.200000,0.004764,1
2015-02-02 14:19:59,23.7180,26.290,578.400000,760.400000,0.004773,1
2015-02-02 14:21:00,23.7300,26.230,572.666667,769.666667,0.004765,1
2015-02-02 14:22:00,23.7225,26.125,493.750000,774.750000,0.004744,1
2015-02-02 14:23:00,23.7540,26.200,488.600000,779.000000,0.004767,1


In [25]:
from keras.optimizers import Adagrad
from tslearn.shapelets import ShapeletModel
from tslearn.shapelets import grabocka_params_to_shapelet_size_dict

from tslearn.preprocessing import TimeSeriesScalerMinMax

In [26]:
from tslearn.preprocessing import TimeSeriesScalerMinMax
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [27]:
#costruisco il set of multivariate time series

samples = np.zeros((257,80,6))
length = 80
# step over the 5,000 in jumps of 200
j = 0
for i in range(0,20560,length):
    samples[j] = df[i:i+length]
    j += 1
print(len(samples),type(samples))

X = np.array(samples)
X.shape

257 <class 'numpy.ndarray'>


(257, 80, 6)

In [28]:
X

array([[[2.37000000e+01, 2.62720000e+01, 5.85200000e+02, 7.49200000e+02,
         4.76416302e-03, 1.00000000e+00],
        [2.37180000e+01, 2.62900000e+01, 5.78400000e+02, 7.60400000e+02,
         4.77266099e-03, 1.00000000e+00],
        [2.37300000e+01, 2.62300000e+01, 5.72666667e+02, 7.69666667e+02,
         4.76515255e-03, 1.00000000e+00],
        ...,
        [2.32900000e+01, 2.85000000e+01, 4.64000000e+02, 1.10800000e+03,
         5.04422920e-03, 1.00000000e+00],
        [2.32150000e+01, 2.86333333e+01, 4.62333333e+02, 1.12783333e+03,
         5.04492265e-03, 1.00000000e+00],
        [2.32000000e+01, 2.86000000e+01, 4.69000000e+02, 1.12875000e+03,
         5.03439915e-03, 1.00000000e+00]],

       [[2.32000000e+01, 2.86500000e+01, 4.69000000e+02, 1.12425000e+03,
         5.04327191e-03, 1.00000000e+00],
        [2.32000000e+01, 2.86200000e+01, 4.69000000e+02, 1.13040000e+03,
         5.03794823e-03, 1.00000000e+00],
        [2.32000000e+01, 2.87000000e+01, 4.64000000e+02, 1.138000

In [29]:
# faccio il clustering sulla X costruita nella cella precedente
from tslearn.piecewise import OneD_SymbolicAggregateApproximation
from tslearn.clustering import TimeSeriesKMeans

n_sax_symbols_avg = 10
n_sax_symbols_slope = 5
d1_sax = OneD_SymbolicAggregateApproximation(
    n_segments=10,
    alphabet_size_avg=n_sax_symbols_avg,
    alphabet_size_slope=n_sax_symbols_slope)
X_d1 = d1_sax.fit_transform(X)

In [30]:
km_d1 = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0)
km_d1.fit(X_d1)

TimeSeriesKMeans(dtw_inertia=False, init='k-means++', max_iter=5,
                 max_iter_barycenter=100, metric='euclidean',
                 metric_params=None, n_clusters=3, n_init=1, n_jobs=None,
                 random_state=0, tol=1e-06, verbose=0)

In [31]:
y = km_d1.labels_
y

array([0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 2, 2, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 0,
       0, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int64)

In [32]:
from sklearn.model_selection import train_test_split, cross_val_score 

from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)
print(X_train.shape, X_test.shape)

(179, 80, 6) (78, 80, 6)


In [34]:
X_test

array([[[2.09633333e+01, 2.50333333e+01, 0.00000000e+00, 5.34000000e+02,
         3.83756341e-03, 0.00000000e+00],
        [2.10000000e+01, 2.50000000e+01, 0.00000000e+00, 5.32000000e+02,
         3.84112401e-03, 0.00000000e+00],
        [2.10000000e+01, 2.50000000e+01, 0.00000000e+00, 5.34500000e+02,
         3.84112401e-03, 0.00000000e+00],
        ...,
        [2.07900000e+01, 2.57633333e+01, 0.00000000e+00, 5.21666667e+02,
         3.90801510e-03, 0.00000000e+00],
        [2.07900000e+01, 2.57450000e+01, 0.00000000e+00, 5.26500000e+02,
         3.90521668e-03, 0.00000000e+00],
        [2.07900000e+01, 2.57950000e+01, 0.00000000e+00, 5.19500000e+02,
         3.91284881e-03, 0.00000000e+00]],

       [[2.10000000e+01, 2.52450000e+01, 0.00000000e+00, 5.74000000e+02,
         3.87900178e-03, 0.00000000e+00],
        [2.10000000e+01, 2.47666667e+01, 0.00000000e+00, 5.71666667e+02,
         3.80505420e-03, 0.00000000e+00],
        [2.10000000e+01, 2.46000000e+01, 0.00000000e+00, 5.710000

In [35]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
scaler = TimeSeriesScalerMeanVariance(mu=0.,std=1.)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [36]:
X_train

array([[[ 0.33211094, -0.63281967,  0.75399493,  1.3418925 ,
         -0.43956835,  0.55809982],
        [ 1.32844377, -0.63281967,  0.69937727,  1.36833477,
         -0.04862772,  0.55809982],
        [ 0.83027736, -0.7634888 ,  0.76992509,  1.32073868,
         -0.36202273,  0.55809982],
        ...,
        [ 0.33211094,  0.88294215, -1.79027784, -1.75714196,
          0.92453087, -1.79179416],
        [ 0.33211094,  0.88294215, -1.79027784, -1.70425742,
          0.92453087, -1.79179416],
        [ 0.33211094,  1.0745902 , -1.81758668, -1.63021905,
          1.09701979, -1.79179416]],

       [[ 1.59637296, -1.77331191,  1.82137546, -0.70519921,
         -2.07270525,  0.        ],
        [ 1.59637296, -1.80112351,  1.63008433, -0.87310778,
         -2.15888369,  0.        ],
        [ 1.59637296, -1.71768871,  1.63008433, -1.25593933,
         -1.90034566,  0.        ],
        ...,
        [-2.18977237,  1.9635559 , -1.16754837,  1.7395496 ,
          1.23165605,  0.        ],
  

In [37]:
X_train = X_train.reshape(179, 80, 6)
X_test = X_test.reshape(78, 80, 6)

In [38]:
n_timesteps, n_outputs, n_features = X_train.shape[1], len(np.unique(y_train)), X_train.shape[2] 
print("TIMESTEPS: ", n_timesteps)
print("N. LABELS: ", n_outputs)
print("N. FEATURES: ", n_features)

TIMESTEPS:  80
N. LABELS:  3
N. FEATURES:  6


In [39]:
X_train_cnn, X_val_cnn, y_train_cnn, y_val_cnn = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train)
X_val_cnn.shape

(36, 80, 6)

In [40]:
X_train

array([[[ 0.33211094, -0.63281967,  0.75399493,  1.3418925 ,
         -0.43956835,  0.55809982],
        [ 1.32844377, -0.63281967,  0.69937727,  1.36833477,
         -0.04862772,  0.55809982],
        [ 0.83027736, -0.7634888 ,  0.76992509,  1.32073868,
         -0.36202273,  0.55809982],
        ...,
        [ 0.33211094,  0.88294215, -1.79027784, -1.75714196,
          0.92453087, -1.79179416],
        [ 0.33211094,  0.88294215, -1.79027784, -1.70425742,
          0.92453087, -1.79179416],
        [ 0.33211094,  1.0745902 , -1.81758668, -1.63021905,
          1.09701979, -1.79179416]],

       [[ 1.59637296, -1.77331191,  1.82137546, -0.70519921,
         -2.07270525,  0.        ],
        [ 1.59637296, -1.80112351,  1.63008433, -0.87310778,
         -2.15888369,  0.        ],
        [ 1.59637296, -1.71768871,  1.63008433, -1.25593933,
         -1.90034566,  0.        ],
        ...,
        [-2.18977237,  1.9635559 , -1.16754837,  1.7395496 ,
          1.23165605,  0.        ],
  

### CNN

In [41]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Conv1D, Conv2D, MaxPool2D, Flatten, Dropout, LeakyReLU, GlobalAveragePooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import regularizers
from keras import optimizers

In [66]:
def build_lstm2(n_timesteps, n_outputs, n_features):
    model = Sequential()
    #LSTM(n_neurons, batch_input_shape=(n_batch, X.shape[1], X.shape[2])
    model.add(LSTM(6, input_shape=(n_timesteps, n_features), return_sequences=True, 
                        kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU()) #activationfunction
    model.add(Dropout(0.8)) #regolarizzazione to avoid overfitting
                            #The term “dropout” refers to dropping out units (hidden and visible) in a neural network.
    
    #1
    for _ in range(2):
        model.add(LSTM(143, kernel_initializer='TruncatedNormal', return_sequences=True))
        model.add(BatchNormalization())#only the first layer of the model requires the input dimension to be explicitly stated; 
                                    #the following layers are able to infer from the previous linear stacked lay
        model.add(LeakyReLU())
        model.add(Dropout(0.05))   

    #2
    model.add(LSTM(143, kernel_initializer='TruncatedNormal', return_sequences=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    
    #3
#    for _ in range(2):
#        model.add(Dense(256, kernel_initializer='TruncatedNormal', kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))) #hidden layer con 256 nodi
#        model.add(BatchNormalization())
#        model.add(LeakyReLU())
#        model.add(Dropout(0.5))
#    #4
#    for _ in range(1):
#        model.add(Dense(64, kernel_initializer='TruncatedNormal'))
#        model.add(BatchNormalization())
#        model.add(LeakyReLU())
#        model.add(Dropout(0.5))
#
#    #5
#    model.add(Dense(32, kernel_initializer='TruncatedNormal'))
#    model.add(BatchNormalization())
#    model.add(LeakyReLU())
#    model.add(Dropout(0.7))
        
    model.add(Dense(n_outputs, activation='sigmoid'))
    adam = optimizers.Adam(lr=1e-5)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [67]:
lstm2 = build_lstm2(n_timesteps, n_outputs, n_features)

In [68]:
lstm2.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 80, 6)             312       
_________________________________________________________________
batch_normalization_27 (Batc (None, 80, 6)             24        
_________________________________________________________________
leaky_re_lu_24 (LeakyReLU)   (None, 80, 6)             0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 80, 6)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 80, 143)           85800     
_________________________________________________________________
batch_normalization_28 (Batc (None, 80, 143)           572       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 80, 143)          

In [51]:
rlr = ReduceLROnPlateau(monitor='accuracy', factor=0.2, patience=10, verbose=0, mode='auto',min_delta=0, cooldown=0, min_lr=0.1,)
mc = ModelCheckpoint('best_model_lstm2.h5', monitor='loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 50
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [70]:
history_lstm2 = lstm2.fit(X_train_cnn, y_train_cnn, epochs=50, batch_size=15, callbacks=callbacks,
                          validation_data=(X_val_cnn, y_val_cnn)).history

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 20s 137ms/step - loss: 0.2912 - accuracy: 0.8741 - val_loss: 1.7223 - val_accuracy: 0.5833
Epoch 2/50
143/143 [==============================] - 14s 95ms/step - loss: 0.4155 - accuracy: 0.8392 - val_loss: 1.0889 - val_accuracy: 0.6111
Epoch 3/50
143/143 [==============================] - 14s 96ms/step - loss: 0.3387 - accuracy: 0.8741 - val_loss: 1.8246 - val_accuracy: 0.5556
Epoch 4/50
143/143 [==============================] - 14s 96ms/step - loss: 0.3776 - accuracy: 0.8252 - val_loss: 1.9890 - val_accuracy: 0.6111
Epoch 5/50
143/143 [==============================] - 12s 84ms/step - loss: 0.3570 - accuracy: 0.8322 - val_loss: 1.6312 - val_accuracy: 0.5833
Epoch 6/50
143/143 [==============================] - 12s 84ms/step - loss: 0.3524 - accuracy: 0.8601 - val_loss: 1.2194 - val_accuracy: 0.6111
Epoch 7/50
143/143 [==============================] - 13s 93ms/step - loss: 0.3004 - accur

In [71]:
y_pred = np.argmax(lstm2.predict(X_test), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.6923076923076923
F1-score [0.63829787 0.84782609 0.        ]
              precision    recall  f1-score   support

           0       0.48      0.94      0.64        16
           1       0.87      0.83      0.85        47
           2       0.00      0.00      0.00        15

    accuracy                           0.69        78
   macro avg       0.45      0.59      0.50        78
weighted avg       0.62      0.69      0.64        78



In [72]:
X_train_cnn.shape

(143, 80, 6)

In [73]:
X_train_cnn2 = X_train_cnn.reshape(X_train_cnn.shape[0], X_train_cnn.shape[1], X_train_cnn.shape[2], 1)
X_val_cnn2 = X_val_cnn.reshape(X_val_cnn.shape[0], X_val_cnn.shape[1], X_val_cnn.shape[2], 1)
X_test_cnn2 = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

X_train_cnn2.shape

(143, 80, 6, 1)

In [74]:
def build_cnn2(n_timesteps, n_features, n_outputs):
    input_shape = (n_timesteps, n_features, 1)

    model = Sequential()
    
    ks1_first = 3
    ks1_second = 3
    
    ks2_first = 4
    ks2_second = 4
    
    model.add(Conv2D(filters=(3), 
                     kernel_size=(ks1_first, ks1_second),
                     input_shape=input_shape, 
                     padding='same',
                     kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.02))
    
    for _ in range(2):
        model.add(Conv2D(filters=(4), 
                     kernel_size= (ks2_first, ks2_second), 
                         padding='same',
                     kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.2))  
    
    model.add(Flatten())
    
    for _ in range(4):
        model.add(Dense(64 , kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.4))
    
    for _ in range(3):
        model.add(Dense(128 , kernel_initializer='TruncatedNormal'))
        model.add(BatchNormalization())
        model.add(LeakyReLU())
        model.add(Dropout(0.3))
  
    model.add(Dense(1024 , kernel_initializer='TruncatedNormal'))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Dropout(0.7))
        
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [75]:
cnn2 = build_cnn2(n_timesteps, n_features, n_outputs)

In [76]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn2.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [77]:
history_cnn2 = cnn2.fit(X_train_cnn2, y_train_cnn, epochs=50, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn2, y_val_cnn)).history

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 12s 83ms/step - loss: 1.2797 - accuracy: 0.4196 - val_loss: 1.0772 - val_accuracy: 0.5833
Epoch 2/50
143/143 [==============================] - 1s 7ms/step - loss: 1.0736 - accuracy: 0.5245 - val_loss: 1.0649 - val_accuracy: 0.5833
Epoch 3/50
143/143 [==============================] - 1s 7ms/step - loss: 1.1755 - accuracy: 0.4895 - val_loss: 1.0569 - val_accuracy: 0.5833
Epoch 4/50
143/143 [==============================] - 1s 7ms/step - loss: 1.0178 - accuracy: 0.5175 - val_loss: 1.0501 - val_accuracy: 0.5833
Epoch 5/50
143/143 [==============================] - 1s 6ms/step - loss: 1.1128 - accuracy: 0.5524 - val_loss: 1.0433 - val_accuracy: 0.5833
Epoch 6/50
143/143 [==============================] - 1s 6ms/step - loss: 1.0164 - accuracy: 0.6084 - val_loss: 1.0405 - val_accuracy: 0.5833
Epoch 7/50
143/143 [==============================] - 1s 6ms/step - loss: 1.0196 - accuracy: 0.5455 -

In [78]:
y_pred = np.argmax(cnn2.predict(X_test_cnn2), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.782051282051282
F1-score [0.66666667 0.96774194 0.        ]
              precision    recall  f1-score   support

           0       0.50      1.00      0.67        16
           1       0.98      0.96      0.97        47
           2       0.00      0.00      0.00        15

    accuracy                           0.78        78
   macro avg       0.49      0.65      0.54        78
weighted avg       0.69      0.78      0.72        78



In [61]:
def build_cnn3(n_timesteps, n_outputs, n_features):
    model = Sequential()
    
    model.add(Conv1D(filters=16, kernel_size=8, activation='relu', input_shape=(n_timesteps, n_features)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(Dropout(0.3))
    
    model.add(GlobalAveragePooling1D())
    
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [62]:
cnn3 = build_cnn3(n_timesteps, n_outputs, n_features)

In [63]:
rlr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)
mc = ModelCheckpoint('best_model_cnn2.h5', monitor='val_loss', save_best_only=True)

callbacks = [rlr, mc]

batch_size = 16
mini_batch_size = int(min(X_train.shape[0]/10, batch_size))

In [64]:
history_cnn3 = cnn3.fit(X_train_cnn, y_train_cnn, epochs=50, batch_size=mini_batch_size, callbacks=callbacks,
                      validation_data=(X_val_cnn, y_val_cnn)).history

Train on 143 samples, validate on 36 samples
Epoch 1/50
143/143 [==============================] - 5s 35ms/step - loss: 0.9788 - accuracy: 0.6224 - val_loss: 1.0445 - val_accuracy: 0.7222
Epoch 2/50
143/143 [==============================] - 1s 4ms/step - loss: 0.8556 - accuracy: 0.7552 - val_loss: 1.0077 - val_accuracy: 0.7500
Epoch 3/50
143/143 [==============================] - 1s 4ms/step - loss: 0.7484 - accuracy: 0.7832 - val_loss: 0.9723 - val_accuracy: 0.7778
Epoch 4/50
143/143 [==============================] - 1s 5ms/step - loss: 0.6800 - accuracy: 0.7692 - val_loss: 0.9492 - val_accuracy: 0.8056
Epoch 5/50
143/143 [==============================] - 1s 6ms/step - loss: 0.6192 - accuracy: 0.8392 - val_loss: 0.9241 - val_accuracy: 0.7778
Epoch 6/50
143/143 [==============================] - 1s 6ms/step - loss: 0.5864 - accuracy: 0.8462 - val_loss: 0.9097 - val_accuracy: 0.7500
Epoch 7/50
143/143 [==============================] - 1s 5ms/step - loss: 0.5201 - accuracy: 0.8392 - 

In [65]:
y_pred = np.argmax(cnn3.predict(X_test), axis=1)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9102564102564102
F1-score [0.89655172 0.95744681 0.78787879]
              precision    recall  f1-score   support

           0       1.00      0.81      0.90        16
           1       0.96      0.96      0.96        47
           2       0.72      0.87      0.79        15

    accuracy                           0.91        78
   macro avg       0.89      0.88      0.88        78
weighted avg       0.92      0.91      0.91        78

